In [ ]:
from utils import get_common_backbone_atoms, generate_df
from Bio import PDB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

In [ ]:

def get_rmsd(path_og, path_m):
    parser = PDB.PDBParser()
    structure_og = parser.get_structure("original", path_og)
    structure_m = parser.get_structure("mutation", path_m)
    atoms_og, atoms_m = get_common_backbone_atoms(structure_og, structure_m)
    superimposer = PDB.Superimposer()
    superimposer.set_atoms(atoms_og, atoms_m)
    superimposer.apply(structure_m.get_atoms())
    
    return superimposer.rms


df_paths = pd.read_csv('../datasets/df_with_scores.csv')



In [ ]:
#strip df_paths columns names
df_paths.columns = df_paths.columns.str.strip()
df_paths.columns

In [ ]:
mutation_path_dict = {} 
for i, row in df_paths.iterrows():
    mutation_path_dict[row['Protein change allele 1']] = row['pdb_file_allele1']
    mutation_path_dict[row['Protein change allele 2']] = row['pdb_file_allele2']

print(len(mutation_path_dict))

In [ ]:
print(mutation_path_dict)
path_mutation_dict = {v:k for k,v in mutation_path_dict.items()}

In [ ]:
paths = list(mutation_path_dict.values())

In [ ]:
# create a df with the pairs path, path 

from itertools import product

pairs = product(paths, paths)

In [ ]:
from tqdm import tqdm

def generate_matriciona(paths):
    with tqdm(total=len(paths)*len(paths)) as pbar:
        M = np.zeros((len(paths), len(paths)))
        for i, path1 in enumerate(paths):
            for j, path2 in enumerate(paths):
                M[i, j] = get_rmsd(path1, path2)
                pbar.update(1)

    return M     

In [ ]:
try:
    matriciona = np.load('../datasets/matriciona.npy')
except:
    print('generating matrix')
    matriciona = generate_matriciona(paths) 
    # the matriciona is symmetric, but due to numerical errors it is not exactly symmetric, we can fix this by averaging the matrix with its transpose
    print(np.linalg.norm(matriciona- matriciona.T))
    np.linalg.norm(matriciona- matriciona.T)
    matriciona = (matriciona + matriciona.T) /2
    print(np.linalg.norm(matriciona- matriciona.T))
    # also the diagonal is not zero, but it should be

    print(np.linalg.norm(np.diag(matriciona)))
    np.fill_diagonal(matriciona, 0)
    print(np.linalg.norm(np.diag(matriciona)))
    np.save('../datasets/matriciona.npy', matriciona)

In [ ]:
plt.imshow(matriciona)

In [ ]:
# for each mutation, compute mean distance to all other mutations
mean_distances = matriciona.mean(axis=1)
std_distances = matriciona.std(axis=1)


In [ ]:
# discard  the mutations with the highest mean distances
threshold = 4
matriciona = matriciona[mean_distances < threshold]
matriciona = matriciona[:, mean_distances < threshold]

In [ ]:
plt.imshow(matriciona, cmap='gray')

## K-medoids
We apply k-medoids algorithm to the protein structures.

In [ ]:
from sklearn_extra.cluster import KMedoids



In [ ]:
sse = []
for k in range(10):
    k_medoids = KMedoids(n_clusters=k+1, random_state=0, metric = 'precomputed').fit(matriciona) 
    sse.append(k_medoids.inertia_)
    print(k_medoids.inertia_)
    

In [ ]:
plt.plot( range(1,11), sse)
# set x ticks to be integers
plt.xticks(np.arange(1, 11, step=1))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.title("SSE vs Number of Clusters")


In [ ]:
# refit with k=2

k_medoids = KMedoids(n_clusters=2, random_state=0, metric = 'precomputed').fit(matriciona)

In [ ]:
# order matrix by cluster
order = np.argsort(k_medoids.labels_)
clusters = k_medoids.labels_
clusters = clusters[order]
matriciona = matriciona[order, :]
matriciona = matriciona[:, order]
# colorbar
plt.imshow(matriciona, cmap='gray')
plt.colorbar()
plt.show()



In [ ]:
# t-SNE
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0, metric='precomputed', init='random')
X_embedded = tsne.fit_transform(matriciona)

plt.scatter(X_embedded[:,0], X_embedded[:,1], c=clusters, cmap='viridis')

## Aggregation and statistics
Now we aggregate the scores by mutation, and see if there is any difference between mutations in the clusters.
From 2 clusters we pass to 3, patients with mutations in the same clusters and patients with two mutations in different clusters.

In [ ]:
def get_cluster(mutation):
    path = mutation_path_dict[mutation]
    idx = paths.index(path)
    return k_medoids.labels_[idx]


df_paths['cluster1'] = df_paths['Protein change allele 1'].apply(get_cluster)
df_paths['cluster2'] = df_paths['Protein change allele 2'].apply(get_cluster)
df_paths['clusters'] = list(zip(df_paths['cluster1'], df_paths['cluster2']))

df_paths.drop(columns=['cluster1', 'cluster2'], inplace=True)
df_paths['clusters'] = df_paths['clusters'].apply(lambda x: str(set(x)))

In [ ]:
# count the number of mutations in each cluster
df_paths['clusters'].value_counts()


In [ ]:
# mean of each metric for each cluster
mean_scores_clusters = df_paths.groupby('clusters').mean(numeric_only=True)


In [ ]:

# std of each metric for each cluster
std_score_clusters = df_paths.groupby('clusters').std(numeric_only=True)


In [ ]:
df_paths.columns    

In [ ]:
columns_to_test = ['birth', 'bmi',
       'physical_health_score', 'mental_health_score', 'AKUSSI_jointpain',
       'AKUSSI_spinalpain', 'KOOSpain', 'KOOSsymptoms', 'KOOSdaily_living',
       'KOOSsport', 'KOOS_QOL', 'HAQ_hapVAS', 'HAQ_haqDI']

for column in columns_to_test:
    cluster1 = df_paths[df_paths['clusters'] == '{0, 1}'][column]
    cluster2 = df_paths[df_paths['clusters'] == '{1}'][column]
    print(column, ttest_ind(cluster1, cluster2))

In [ ]:
for column in columns_to_test:
    cluster1 = df_paths[df_paths['clusters'] == '{0}'][column]
    cluster2 = df_paths[df_paths['clusters'] == '{1}'][column]
    print(column, ttest_ind(cluster1, cluster2))

In [ ]:
for column in columns_to_test:
    cluster1 = df_paths[df_paths['clusters'] == '{0, 1}'][column]
    cluster2 = df_paths[df_paths['clusters'] == '{0}'][column]
    print(column, ttest_ind(cluster1, cluster2))

In [ ]:
# plot values of each metric for each cluster
import seaborn as sns

fig, ax = plt.subplots(4, 4, figsize=(20, 20))

for i, column in enumerate(columns_to_test):
    sns.boxplot(x='clusters', y=column, data=df_paths, ax=ax[i//4, i%4])
    ax[i//4, i%4].set_title(column)
    ax[i//4, i%4].set_ylabel('')
    ax[i//4, i%4].set_xlabel('')
    ax[i//4, i%4].set_xlabel('Clusters')
    ax[i//4, i%4].set_ylabel(column)
    ax[i//4, i%4].set_title(column)
plt.tight_layout()
plt.show()
    

In [ ]:
# plot distribution of each metric for each cluster

fig, ax = plt.subplots(4, 4, figsize=(20, 20))

for i, column in enumerate(columns_to_test):
    sns.histplot(df_paths, x=column, hue='clusters', ax=ax[i//4, i%4], kde=True)
    ax[i//4, i%4].set_title(column)
    ax[i//4, i%4].set_ylabel('')
    ax[i//4, i%4].set_xlabel('')
    ax[i//4, i%4].set_xlabel(column)
    ax[i//4, i%4].set_title(column)
plt.tight_layout()
plt.show()

In [ ]:
for column in columns_to_test:
    cluster1 = df_paths[(df_paths['clusters'] == '{0, 1}') | (df_paths['clusters']=='{0}')][column]
    cluster2 = df_paths[df_paths['clusters'] == '{1}'][column]
    print(column, ttest_ind(cluster1, cluster2))

In [ ]:
# plot values of each metric for each cluster with 2 clusters
import seaborn as sns

clusters = df_paths['clusters'].apply(lambda x: x[1:-1].split(', '))
df_paths['cluster1'] = clusters.apply(lambda x: int(x[0]))

fig, ax = plt.subplots(4, 4, figsize=(20, 20))

for i, column in enumerate(columns_to_test):
    sns.boxplot(x='cluster1', y=column, data=df_paths, ax=ax[i//4, i%4])
    ax[i//4, i%4].set_title(column)
    ax[i//4, i%4].set_ylabel('')
    ax[i//4, i%4].set_xlabel('')
    ax[i//4, i%4].set_xlabel('Clusters')
    ax[i//4, i%4].set_ylabel(column)
    ax[i//4, i%4].set_title(column)

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(20, 20))

for i, column in enumerate(columns_to_test):
    sns.histplot(df_paths, x=column, hue='cluster1', ax=ax[i//4, i%4], kde=True)
    ax[i//4, i%4].set_title(column)
    ax[i//4, i%4].set_ylabel('')
    ax[i//4, i%4].set_xlabel('')
    ax[i//4, i%4].set_xlabel(column)
    ax[i//4, i%4].set_title(column)

In [ ]:
df = pd.read_excel('../datasets/aku_prin_v2.0.xlsx')

# strip column names

df.columns = df.columns.str.strip()
df_paths.columns = df_paths.columns.str.strip()
df_paths.columns

In [ ]:
for col in df.columns:
    # if column is in df_paths
    if col in df_paths.columns and col !='patient':
        # drop from df_complete
        print(col)
        df.drop(columns=[col], inplace=True)
        

In [ ]:
df_complete = df_paths.merge(df, on='patient', how='inner')
# remove duplicate columns


In [ ]:
df_complete.shape

In [ ]:
df_complete.columns 

In [ ]:
df_complete_cl = df_complete.loc[:, df_complete.isna().mean() < 1]

In [ ]:
df_complete = df_complete_cl

In [ ]:
df_complete.head()

In [ ]:
# statistical tests on the new columns
from scipy.stats import ttest_ind
for column in df_complete.columns:
    # if column is numeric
    if df_complete[column].dtype == 'float64':
        cluster1 = df_complete[(df_complete['clusters'] == '{0}') | (df_complete['clusters']=='{0, 1}')][column]
        cluster2 = df_complete[df_complete['clusters'] == '{1}'][column]
        # print if p-value is significant
        result = ttest_ind(cluster1, cluster2)
        if result.pvalue < 1:
            print(column, result)


In [ ]:
df.head()

In [ ]:
# read other sheets
df = pd.read_excel('../datasets/aku_prin_v2.0.xlsx')
df_plasma = pd.read_excel('../datasets/aku_prin_v2.0.xlsx', sheet_name='plasma')
df_piscio = pd.read_excel('../datasets/aku_prin_v2.0.xlsx', sheet_name='URINE')


In [ ]:
# strip column names
df_plasma.columns = df_plasma.columns.str.strip()
df_piscio.columns = df_piscio.columns.str.strip()

In [ ]:
#v rename P.code to patient
df_plasma.rename(columns={'P.code':'patient'}, inplace=True)
df_piscio.rename(columns={'P.Code':'patient'}, inplace=True)

In [ ]:
df=df.merge(df_plasma, on='patient', how='left')
df=df.merge(df_piscio, on='patient', how='left')

In [ ]:
unit_measure = 'mmoli/l'

df_piscio = df_piscio[df_piscio['u.m.'] == unit_measure]


In [ ]:
df_plasma